### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re
import unicodedata
import urllib.parse

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [4]:
# Set the base URL
nyt_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# URL-encode the filter query
encoded_filter_query = urllib.parse.quote(filter_query)

# Build URL
query_url = f"{nyt_url}api-key={nyt_api_key}&fq={encoded_filter_query}\
            &sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"

In [5]:
# Test Query URL
response = requests.get(query_url)
response

<Response [200]>

In [7]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):
    query_url = f"{nyt_url}api-key={nyt_api_key}&fq={encoded_filter_query}\
                &sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"
    # create query with a page number
    query_url = f"{query_url}&page={str(page)}"
    # Make a "GET" request and retrieve the JSON
    articles = requests.get(query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in articles['response']['docs']:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Page {page} retrieved successfully.")

        # Print the page number that had no results then break from the loop
        if not articles['response']['docs']:
            print(f"Page {page} not found. Exiting loop.")
            break
    except KeyError:
        print(f"Error retrieving date for page {page}. Exiting loop.")
        break


Page 0 retrieved successfully.
Page 1 retrieved successfully.
Page 2 retrieved successfully.
Page 3 retrieved successfully.
Page 4 retrieved successfully.
Page 5 retrieved successfully.
Page 6 retrieved successfully.
Page 7 retrieved successfully.
Page 8 retrieved successfully.
Page 9 retrieved successfully.
Page 10 retrieved successfully.
Page 11 retrieved successfully.
Page 12 retrieved successfully.
Page 13 retrieved successfully.
Page 14 retrieved successfully.
Page 15 retrieved successfully.
Page 16 retrieved successfully.
Page 17 retrieved successfully.
Page 18 retrieved successfully.
Page 19 retrieved successfully.


In [8]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [9]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [10]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 

# Function to extract the title from headline.main column
def extract_title(text):
    match = re.search(r"\u2018(.*?)\u2019", text)
    if match:
        return match.group(1) # return the text between the single quotes
    else:
        return "" # Return empty string if no match is found

reviews_df['title'] = reviews_df['headline.main'].apply(extract_title)
reviews_df[['headline.main', 'title']]

,headline.main,title
0,"‘The Attachment Diaries’ Review: Love, Sick",The Attachment Diaries
1,Review: ‘What’s Love Got to Do With It?’ Proba...,What
2,‘You Can Live Forever’ Review: Do You Love Me ...,You Can Live Forever
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,A Tourist
4,‘Other People’s Children’ Review: True Romance,Other People
...,...,...
195,Review: A Combustible Pair Find Love in ‘The O...,The Other Half
196,"Review: Love as the World Wars, in ‘The Ottoma...",The Ottoman Lieutenant
197,Review: It’s All Mirth and Taxes in ‘Love & Ta...,Love & Taxes
198,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...","Everybody Loves Somebody,"


In [11]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df.head()



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People


In [12]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
nyt_titles_list = reviews_df['title'].to_list()
print(nyt_titles_list)
print(len(nyt_titles_list))

['The Attachment Diaries', 'What', 'You Can Live Forever', 'A Tourist', 'Other People', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story', 'Aulcie', 'Love Is Love Is Love', 'Love Hard', 'Bergman Island', 'Hard Luc

### Access The Movie Database API

In [13]:
# Prepare The Movie Database query
tmdb_url = "https://api.themoviedb.org/3/search/movie?query="

tmdb_key_string = "&api_key=" + tmdb_api_key


In [14]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0  # Start from 0 because we increment before checking

# Loop through the titles
for title in nyt_titles_list:
    # Increment the request counter for each title
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    search_response = requests.get(tmdb_url + title + tmdb_key_string)
    
    if search_response.status_code == 200:
        # Include a try clause to search for the full movie details.
        try:
            search_data = search_response.json()
            
            if search_data['results']:
                # Get movie id
                movie_id = search_data['results'][0]['id']
                
                # Make a request for the full movie details
                movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
                movie_response = requests.get(movie_details_url)
                
                if movie_response.status_code == 200:
                    movie_details = movie_response.json()
                    
                    # Extract the genre names into a list
                    genres = [genre['name'] for genre in movie_details.get('genres', [])]
                    
                    # Extract the spoken_languages' English name into a list
                    spoken_languages = [language['english_name'] for language in movie_details.get('spoken_languages', [])]
                    
                    # Extract the production_countries' name into a list
                    production_countries = [country['name'] for country in movie_details.get('production_countries', [])]

                    # Add the relevant data to a dictionary and append it to the tmdb_movies_list
                    movie_data = {
                        'title': movie_details['title'],
                        'original title': movie_details['original_title'],
                        'budget': movie_details['budget'],
                        'original language': movie_details['original_language'],
                        'homepage': movie_details['homepage'],
                        'overview': movie_details['overview'],
                        'popularity': movie_details['popularity'],
                        'runtime': movie_details['runtime'],
                        'revenue': movie_details['revenue'],
                        'release date': movie_details['release_date'],
                        'vote average': movie_details['vote_average'],
                        'vote count': movie_details['vote_count'],
                        'genres': genres,
                        'spoken_languages': spoken_languages,
                        'production_countries': production_countries
                    }
                    tmdb_movies_list.append(movie_data)
                    print(f"Found: {movie_details['title']}")
                    
        except Exception as e:
            print(f"Error processing {title}: {e}")
            
    else:
        print(f"Failed to find {title}. Status code: {search_response.status_code}")
    
    # Check if we need to sleep after every 50 requests
    if request_counter % 50 == 0:
        print(f"Sleeping for API Call Limit after {request_counter} requests")
        time.sleep(1)  # Sleep for 1 second

# Print or use tmdb_movies_list after the loop
print(tmdb_movies_list)


Found: The Attachment Diaries
Found: What...
Found: You Can Live Forever
Found: A Tourist's Guide to Love
Found: Other People
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: Pamela, A Love Story
Found: In from the Side
Found: After Love
Found: Alcarràs
Found: Nelly and Monsieur Arnaud
Found: Lady Chatterley
Found: The Sound of Christmas
Found: The Inspection
Found: Bones and All
Found: My Policeman
Found: About Fate
Found: Waiting for Bojangles
Found: I Love My Dad
Found: A Love Song
Found: Alone Together
Found: Art of Love
Found: 2099: The Soldier Protocol
Found: Thor: Love and Thunder
Found: Both Sides of the Blade
Found: Fire of Love
Found: Art of Love
Found: Stay Prayed Up
Found: Benediction
Found: Dinner in America
Found: In a New York Minute
Found: Anaïs in Love
Found: I Love America
Found: See You Then
Found: La Mami
Found: After Love
Found: Deep Water
Found: Lucy and Desi


In [15]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list, indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original title": "El apego",
        "budget": 0,
        "original language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.409,
        "runtime": 102,
        "revenue": 0,
        "release date": "2021-10-07",
        "vote average": 3.0,
        "vote count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [16]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original title,budget,original language,homepage,overview,popularity,runtime,revenue,release date,vote average,vote count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.409,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What...,What...,0,en,,"It's another ordinary day for Ricky, until he ...",2.905,0,0,2024-07-11,0.000,0,"[Comedy, Drama]",[English],[Australia]
2,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",47.127,96,15055,2023-03-24,6.600,37,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
3,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",33.194,96,0,2023-04-21,6.287,162,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
4,Other People,Other People,0,en,,"David, a struggling comedy writer fresh off fr...",13.635,97,93000,2016-09-09,6.300,193,"[Comedy, Drama]",[English],[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,8.369,103,0,2016-12-02,6.300,24,"[Drama, Romance]",[English],[Canada]
194,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",25.914,111,413844,2017-03-10,6.164,247,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]"
195,Art of Love,Romantik Hırsız,0,tr,https://www.netflix.com/title/81689799,After learning that the art thief she has been...,82.424,99,0,2024-03-13,6.527,112,"[Romance, Action, Drama]",[Turkish],[Turkey]
196,Everybody Loves Somebody,Everybody Loves Somebody,2000000,en,,"On the surface, Clara Barron seems to have it ...",15.606,102,0,2017-02-10,6.538,159,"[Romance, Comedy]","[English, Spanish]",[Mexico]


### Merge and Clean the Data for Export

In [17]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df, reviews_df, on='title', how='inner')
merged_df

,title,original title,budget,original language,homepage,overview,popularity,runtime,revenue,release date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.409,102,0,2021-10-07,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",47.127,96,15055,2023-03-24,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,Other People,Other People,0,en,,"David, a struggling comedy writer fresh off fr...",13.635,97,93000,2016-09-09,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
3,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,19.645,100,37820,2023-04-07,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
4,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,6.754,95,77145,2023-04-13,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Hounds of Love,Hounds of Love,0,en,http://www.houndsoflovemovie.com/,When Vicki Maloney is randomly abducted from a...,22.554,108,0,2016-10-07,...,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
147,The Promise,The Promise,90000000,en,http://www.survivalpictures.org/the-promise/,Set during the last days of the Ottoman Empire...,18.720,133,12448676,2016-12-02,...,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
148,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,8.369,103,0,2016-12-02,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
149,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",25.914,111,413844,2017-03-10,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None


In [18]:
# Remove list brackets and quotation marks on the columns containing lists

# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')
    

# Display the fixed DataFrame
merged_df[['title', 'original title', 'genres', 'spoken_languages', 'production_countries']]

,title,original title,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,You Can Live Forever,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America"
2,Other People,Other People,"Comedy, Drama",English,United States of America
3,One True Loves,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America"
4,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,Documentary,English,United States of America
...,...,...,...,...,...
146,Hounds of Love,Hounds of Love,"Crime, Drama, Horror",English,Australia
147,The Promise,The Promise,"History, Romance, Drama","Armenian, German, English, French","Spain, United States of America"
148,The Other Half,The Other Half,"Drama, Romance",English,Canada
149,The Ottoman Lieutenant,The Ottoman Lieutenant,"Romance, Drama, War","Turkish, English","Turkey, United States of America"


In [19]:
# Drop "byline.person" column
merged_df.drop(columns=['byline.person'], inplace=True)
merged_df


,title,original title,budget,original language,homepage,overview,popularity,runtime,revenue,release date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.409,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",47.127,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,Other People,Other People,0,en,,"David, a struggling comedy writer fresh off fr...",13.635,97,93000,2016-09-09,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
3,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,19.645,100,37820,2023-04-07,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
4,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,6.754,95,77145,2023-04-13,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Hounds of Love,Hounds of Love,0,en,http://www.houndsoflovemovie.com/,When Vicki Maloney is randomly abducted from a...,22.554,108,0,2016-10-07,...,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,None,None,None,By Ben Kenigsberg,None
147,The Promise,The Promise,90000000,en,http://www.survivalpictures.org/the-promise/,Set during the last days of the Ottoman Empire...,18.720,133,12448676,2016-12-02,...,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,None
148,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,8.369,103,0,2016-12-02,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
149,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",25.914,111,413844,2017-03-10,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None


In [20]:
# Delete duplicate rows and reset index
movies_collected_df = merged_df.drop_duplicates().reset_index(drop=True)
movies_collected_df

,title,original title,budget,original language,homepage,overview,popularity,runtime,revenue,release date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.409,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",47.127,96,15055,2023-03-24,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,Other People,Other People,0,en,,"David, a struggling comedy writer fresh off fr...",13.635,97,93000,2016-09-09,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
3,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,19.645,100,37820,2023-04-07,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
4,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,6.754,95,77145,2023-04-13,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,"Everything, Everything","Everything, Everything",10000000,en,http://everythingeverythingmovie.com,A teenager who's lived a sheltered life becaus...,32.561,96,61621140,2017-05-18,...,270,"Review: ‘Everything, Everything’ Pits Love Aga...",None,None,"Everything, Everything",None,None,None,By Neil Genzlinger,None
138,Hounds of Love,Hounds of Love,0,en,http://www.houndsoflovemovie.com/,When Vicki Maloney is randomly abducted from a...,22.554,108,0,2016-10-07,...,280,Review: ‘Hounds of Love’ Is Tense and Deadly D...,None,None,Hounds of Love,None,None,None,By Ben Kenigsberg,None
139,The Promise,The Promise,90000000,en,http://www.survivalpictures.org/the-promise/,Set during the last days of the Ottoman Empire...,18.720,133,12448676,2016-12-02,...,455,Review: ‘The Promise’ Finds a Love Triangle in...,None,None,"A Love Triangle, Near the Front Lines",None,None,None,By Jeannette Catsoulis,None
140,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,8.369,103,0,2016-12-02,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None


In [21]:
# Export data to CSV without the index
movies_collected_df.to_csv('movies_collected.csv', index=False)